In [1]:
import sys
sys.version

'3.10.6 (tags/v3.10.6:9c7b4bd, Aug  1 2022, 21:53:49) [MSC v.1932 64 bit (AMD64)]'

In [2]:
! pip install -q python-dotenv
! pip install -q neo4j
! pip install -q langchain
! pip install -q langchain-openai


[notice] A new release of pip available: 22.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip available: 22.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip available: 22.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip available: 22.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
from neo4j import GraphDatabase
import json

from langchain_community.graphs import Neo4jGraph
from langchain_community.vectorstores import Neo4jVector
from langchain_openai import OpenAIEmbeddings




import sys
sys.path.append('../utils')  
from helper import chunk_doc  

In [4]:
import dotenv
import os

dotenv.load_dotenv()

os.environ["NEO4J_URI"] = os.getenv("URL")
os.environ["NEO4J_USERNAME"]= os.getenv("USERNAME")
os.environ["NEO4J_PASSWORD"] = os.getenv("PASSWORD2")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAIKEY")

In [5]:
#
# Neo4j with LangChain wrapper
# [ERROR RESOLVED w/ APOC PLUGIN ENABLED]
#

graphDB = Neo4jGraph()

In [ ]:
# graphDB.query("""
# MATCH (n)
# RETURN COUNT(n)
# """)

In [7]:
#
# ROOT NODES
#

root_aacsb_query =""" 
MERGE (r:Root {nodeCat: 'AACSB'})
SET r.Title = 'AACSB',
    r.nodeType = 'ROOT'
RETURN r, ID(r) AS elementID
"""

root_institution_query =""" 
MERGE (r:Root {nodeCat: 'INSTITUTION'})
SET r.Title = 'INSTITUTION',
    r.nodeType = 'ROOT'
RETURN r, ID(r) AS elementID
"""

In [8]:
def create_root_nodes():
    print('Initializing AACSB and INSTITUTION Root Nodes')
    try:
        graphDB.query(root_aacsb_query)
        graphDB.query(root_institution_query)
    
    except Exception as e:
        raise e


In [45]:
create_root_nodes()

Initializing AACSB and INSTITUTION Root Nodes


In [10]:
#
# STANDARDS NODES and child nodes
# @TODO Link by -- parentStandardNum
#

standards_query = """
MERGE(s:Standard {standardNum: $standardsIn.Standard_num})
    ON CREATE SET 
        s.parentSectionNum = $standardsIn.Section,
        s.title =  $standardsIn.Standard_title,
        s.text =  $standardsIn.Standard_formal, 
        s.nodeType = 'STANDARD', 
        s.nodeCat = 'AACSB',
        s.source = 'https://www.aacsb.edu/educators/accreditation/business-accreditation/aacsb-business-accreditation-standards',
        s.year = 2020
        
RETURN s, ID(s) as elementId
"""

formal_query = """
MERGE(f:Formal {parentStandardNum: $standardsIn.Standard_num})
    ON CREATE SET 
        f.text = $standardsIn.Standard_formal,
        f.standardTitle =  $standardsIn.Standard_title,
        f.nodeType = 'FORMAL', 
        f.nodeCat = 'AACSB'
RETURN f, ID(f) as elementId
"""

definitions_query = """
MERGE(d:Definitions {parentStandardNum: $standardsIn.Standard_num})
    ON CREATE SET 
        d.text = $standardsIn.Definitions,
        d.standardTitle =  $standardsIn.Standard_title,
        d.nodeType = 'DEFINITIONS', 
        d.nodeCat = 'AACSB'
RETURN d, ID(d) as elementId
"""

basis_query = """
MERGE(b:Basis {parentStandardNum: $standardsIn.Standard_num})
    ON CREATE SET 
        b.text = $standardsIn.Basis_for_judgement,
        b.standardTitle =  $standardsIn.Standard_title,
        b.nodeType = 'BASIS', 
        b.nodeCat = 'AACSB'
RETURN b, ID(b) as elementId
"""

docs_query = """
MERGE(d:Documentation {parentStandardNum: $standardsIn.Standard_num})
    ON CREATE SET 
        d.text = $standardsIn.Supporting_docs,
        d.standardTitle =  $standardsIn.Standard_title,
        d.nodeType = 'DOCUMENTATION', 
        d.nodeCat = 'AACSB'
RETURN d, ID(d) as elementId
"""



In [11]:
#
# Read in Standards JSON
#

path =r'..\data\aacsb\processed\STANDARDS_2024-04-04_13-21-46.json'
json_file = json.load(open(path))
print(type(json_file)) #list

<class 'list'>


In [ ]:
# standard_1 = json_file[0]
# print(type(standard_1))

# for k,v in standard_1.items():
#     print(k,type(v))

In [ ]:
# print(standard_1['Standard_num'])

In [ ]:
# basis = standard_1['Basis_for_judgement']
# print(type(basis))
# print(basis)
# print(len(basis))

In [ ]:
#
# testing split function
#


#result = split_text_from_node(basis)
# result = chunk_doc(json_file[0])

# print(type(result))
# print(result)

In [ ]:
# for k,v in result.items():
#     print(k,v)

In [12]:
#
# General queries 
#

datachunk_query = """
UNWIND $chunksIn AS chunk
MERGE (c:Chunk {
    UUID: apoc.create.uuid(),
    parentStandardNum: $standardsIn.Standard_num,
    text: chunk.text,
    embedding: 0,
    standardTitle: $standardsIn.Standard_title,
    nodeType: 'DATACHUNK',
    parentType: chunk.parentType,
    parentId: $parentId,
    nodeCat: 'AACSB',
    chunkSeqNum: chunk.chunk_sequence_id
})
RETURN c
"""

# Create Links
linking_query = """
  MATCH (from_same_section:Chunk)
  WHERE from_same_section.parentId = $parentId
    AND from_same_section.parentType = $parentType
  WITH from_same_section
    ORDER BY from_same_section.chunkSeqNum ASC
  WITH collect(from_same_section) as section_chunk_list
    CALL apoc.nodes.link(
        section_chunk_list, 
        "NEXT", 
        {avoidDuplicates: true}
    )  // NEW!!!
  RETURN size(section_chunk_list)
"""

In [ ]:
#
# Standard
#


# db_result = graphDB.query(standards_query,
#               params={'standardsIn':standard_1})

In [ ]:
#db_result[0]['s']['nodeType']
#db_result

In [ ]:
# #
# # Standard: Data Chunking
# #
# graphDB.query(datachunk_query,
#                 params={
#                     'standardsIn':standard_1,
#                     'chunksIn':result['Standard_formal'],
#                     'parentId':db_result[0]['elementId']
#                 })

In [13]:


#
# Standard: Link Chunks to Parent
#

link_to_parent_standard = """
MATCH (c:Chunk), (n: Standard)
WHERE c.parentId = $parentId
    AND c.parentType = n.nodeType
    AND c.parentStandardNum = n.standardNum //new
MERGE (c)-[newRelationship:PART_OF]->(n)
RETURN count(newRelationship) AS num_links_created
"""

#
# Link first chunk node as head
#


link_first_to_parent_standard = """
MATCH (c:Chunk), (n:Standard)
WHERE c.parentId = $parentId
  AND c.parentType = n.nodeType
  AND c.chunkSeqNum = 0
  AND c.parentStandardNum = n.standardNum //new to fix
MERGE (c)-[newRelationship:FIRST_CHUNK_NODE]->(n)
RETURN count(newRelationship) AS num_links_created
"""

In [ ]:
#
# Standard Chunk Linking
#


# graphDB.query(linking_query, 
#               params={
#                   'parentId':db_result[0]['elementId'],
#                   'parentType':db_result[0]['s']['nodeType']
#               })

In [ ]:
# graphDB.query(link_to_parent_standard, 
#               params={
#                   'parentId':db_result[0]['elementId']
#               })

In [ ]:
# graphDB.query(link_first_to_parent_standard,params={
#     'parentId':db_result[0]['elementId']
# } )

In [ ]:
#
# Create Standard Child Node 1: 
#

# def_result = graphDB.query(definitions_query,
#               params={'standardsIn':standard_1})

In [ ]:
# print(len(result['Definitions']))
# print(def_result[0]['elementId'])

In [ ]:
# graphDB.query(datachunk_query,
#                 params={
#                     'standardsIn':standard_1,
#                     'chunksIn':result['Definitions'],
#                     'parentId':def_result[0]['elementId']
#                 })

In [ ]:
# print(def_result[0]['elementId'])
# print(def_result[0]['d']['nodeType'])

In [ ]:
# #
# # Definitions Chunk Linking
# #

# graphDB.query(linking_query, 
#               params={
#                   'parentId':def_result[0]['elementId'],
#                   'parentType':def_result[0]['d']['nodeType']
#               })

In [14]:
#
# Formal Queries
#



link_to_parent_formal = """
MATCH (c:Chunk), (n: Formal)
WHERE c.parentId = $parentId 
    AND c.parentType = n.nodeType
    AND c.parentStandardNum = n.parentStandardNum //new
MERGE (c)-[newRelationship:PART_OF_FORMAL]->(n)
RETURN count(newRelationship) AS num_links_created
"""

#
# Link first chunk node as head to definiton
#


link_first_to_parent_formal = """
MATCH (c:Chunk), (n:Formal)
WHERE c.parentId = $parentId
  AND c.parentType = n.nodeType
  AND c.chunkSeqNum = 0
  AND c.parentStandardNum = n.parentStandardNum //new
MERGE (c)-[newRelationship:FIRST_CHUNK_NODE]->(n)
RETURN count(newRelationship) AS num_links_created
"""

#
# Link Def to Standard
#

link_formal_to_standard = """
MATCH (f: Formal), (s:Standard)
WHERE f.parentStandardNum = s.standardNum
    AND f.standardTitle = s.title
MERGE (f)-[newRelationship: FORMAL_DESCRIPTION_OF]->(s)
RETURN count(newRelationship) AS num_links_created
"""

In [15]:
#
# Definition Queries
#



link_to_parent_def = """
MATCH (c:Chunk), (n: Definitions)
WHERE c.parentId = $parentId 
    AND c.parentType = n.nodeType
    AND c.parentStandardNum = n.parentStandardNum //new
MERGE (c)-[newRelationship:PART_OF_DEF]->(n)
RETURN count(newRelationship) AS num_links_created
"""

#
# Link first chunk node as head to definiton
#


link_first_to_parent_def = """
MATCH (c:Chunk), (n:Definitions)
WHERE c.parentId = $parentId
  AND c.parentType = n.nodeType
  AND c.chunkSeqNum = 0
  AND c.parentStandardNum = n.parentStandardNum //new
MERGE (c)-[newRelationship:FIRST_CHUNK_NODE]->(n)
RETURN count(newRelationship) AS num_links_created
"""

#
# Link Def to Standard
#

link_def_to_standard = """
MATCH (d: Definitions), (s:Standard)
WHERE d.parentStandardNum = s.standardNum
    AND d.standardTitle = s.title
MERGE (d)-[newRelationship: TERM_DEFINITIONS_FOR]->(s)
RETURN count(newRelationship) AS num_links_created
"""

In [ ]:
# #
# # Link chunks to Definitions (parent)
# #


# graphDB.query(link_to_parent_def, 
#               params={
#                   'parentId':def_result[0]['elementId']
#               })

In [ ]:
# graphDB.query(link_first_to_parent_def,params={
#     'parentId':def_result[0]['elementId']
# } )

In [ ]:
# graphDB.query(link_def_to_standard)

In [16]:
#
# Basis Queries
#

link_to_parent_basis = """
MATCH (c:Chunk), (n: Basis)
WHERE c.parentId = $parentId 
    AND c.parentType = n.nodeType
    AND c.parentStandardNum = n.parentStandardNum //new
MERGE (c)-[newRelationship:PART_OF_BASIS]->(n)
RETURN count(newRelationship) AS num_links_created
"""


link_first_to_parent_basis = """
MATCH (c:Chunk), (n:Basis)
WHERE c.parentId = $parentId
  AND c.parentType = n.nodeType
  AND c.chunkSeqNum = 0
  AND c.parentStandardNum = n.parentStandardNum //new
MERGE (c)-[newRelationship:FIRST_CHUNK_NODE]->(n)
RETURN count(newRelationship) AS num_links_created
"""

link_basis_to_standard = """
MATCH (d: Basis), (s:Standard)
WHERE d.parentStandardNum = s.standardNum
  AND d.standardTitle = s.title
MERGE (d)-[newRelationship: BASIS_OF]->(s)
RETURN count(newRelationship) AS num_links_created
"""

In [ ]:
# #
# # Basis Process
# #

# basis_result = graphDB.query(basis_query,
#               params={'standardsIn':standard_1})

# graphDB.query(datachunk_query,
#                 params={
#                     'standardsIn':standard_1,
#                     'chunksIn':result['Basis_for_judgement'],
#                     'parentId':basis_result[0]['elementId']
#                 })

# graphDB.query(linking_query, 
#               params={
#                   'parentId':basis_result[0]['elementId'],
#                   'parentType':basis_result[0]['b']['nodeType']
#               })


# #make custom query
# graphDB.query(link_to_parent_basis, 
#               params={
#                   'parentId':basis_result[0]['elementId']
#               })

# graphDB.query(link_first_to_parent_basis,params={
#     'parentId':basis_result[0]['elementId']
# } )

# graphDB.query(link_basis_to_standard)

In [17]:
#
# Docs queries
#



link_to_parent_docs = """
MATCH (c:Chunk), (n: Documentation)
WHERE c.parentId = $parentId 
    AND c.parentType = n.nodeType
    AND c.parentStandardNum = n.parentStandardNum //new
MERGE (c)-[newRelationship:PART_OF_DOCS]->(n)
RETURN count(newRelationship) AS num_links_created
"""


link_first_to_parent_docs = """
MATCH (c:Chunk), (n:Documentation)
WHERE c.parentId = $parentId
  AND c.parentType = n.nodeType
  AND c.chunkSeqNum = 0
  AND c.parentStandardNum = n.parentStandardNum //new
MERGE (c)-[newRelationship:FIRST_CHUNK_NODE]->(n)
RETURN count(newRelationship) AS num_links_created
"""

link_docs_to_standard = """
MATCH (d: Documentation), (s:Standard)
WHERE d.parentStandardNum = s.standardNum
  AND d.standardTitle = s.title
MERGE (d)-[newRelationship: DOCUMENTS]->(s)
RETURN count(newRelationship) AS num_links_created
"""

In [ ]:
# #
# # Documentation Node Process
# #
# docs_result = graphDB.query(docs_query,
#               params={'standardsIn':standard_1})


# graphDB.query(datachunk_query,
#                 params={
#                     'standardsIn':standard_1,
#                     'chunksIn':result['Supporting_docs'],
#                     'parentId':docs_result[0]['elementId']
#                 })

# graphDB.query(linking_query, 
#               params={
#                   'parentId':docs_result[0]['elementId'],
#                   'parentType':docs_result[0]['d']['nodeType']
#               })


# #make custom query
# graphDB.query(link_to_parent_docs, 
#               params={
#                   'parentId':docs_result[0]['elementId']
#               })

# graphDB.query(link_first_to_parent_docs,params={
#     'parentId':docs_result[0]['elementId']
# } )

# graphDB.query(link_docs_to_standard)

In [35]:
def create_standards_node(standard, result):
    print(f"Initializing standard nodes for: \n Standard Num: {standard['Standard_num']} \n Standard Title:{standard['Standard_title']} ")

    try:
        standard_result = graphDB.query(standards_query,
                    params={'standardsIn':standard})


        # print(graphDB.query(datachunk_query,
        #             params={
        #                 'standardsIn':standard,
        #                 'chunksIn':result['Standard_formal'],
        #                 'parentId':standard_result[0]['elementId']
        #             }))

        # print(graphDB.query(linking_query, 
        #             params={
        #                 'parentId':standard_result[0]['elementId'],
        #                 'parentType':standard_result[0]['s']['nodeType']
        #             }))


        # print(graphDB.query(link_to_parent_standard, 
        #             params={
        #                 'parentId':standard_result[0]['elementId']
        #             }))

        # print(graphDB.query(link_first_to_parent_standard,
        #             params={
        #                 'parentId':standard_result[0]['elementId']
        #             }))
        #TODO link standards to SECTIONS
        #print(graphDB.query(link_docs_to_standard))

    except Exception as e:
        raise e 

In [ ]:
#------

In [33]:

def create_formal_nodes(standard, result):
    print(f"Initializing formal nodes for: \n Standard Num: {standard['Standard_num']} \n Standard Title:{standard['Standard_title']} ")

    try:
        formal_result = graphDB.query(formal_query,
                    params={'standardsIn':standard})


        print(graphDB.query(datachunk_query,
                    params={
                        'standardsIn':standard,
                        'chunksIn':result['Standard_formal'],
                        'parentId':formal_result[0]['elementId']
                    }))

        print(graphDB.query(linking_query, 
                    params={
                        'parentId':formal_result[0]['elementId'],
                        'parentType':formal_result[0]['f']['nodeType']
                    }))


        print(graphDB.query(link_to_parent_formal, 
                    params={
                        'parentId':formal_result[0]['elementId']
                    }))

        print(graphDB.query(link_first_to_parent_formal,
                    params={
                        'parentId':formal_result[0]['elementId']
                    }))

        print(graphDB.query(link_formal_to_standard))

    except Exception as e:
        raise e 

In [20]:

def create_docs_nodes(standard, result):
    print(f"Initializing documentation nodes for: \n Standard Num: {standard['Standard_num']} \n Standard Title:{standard['Standard_title']} ")

    try:
        docs_result = graphDB.query(docs_query,
                    params={'standardsIn':standard})


        print(graphDB.query(datachunk_query,
                    params={
                        'standardsIn':standard,
                        'chunksIn':result['Supporting_docs'],
                        'parentId':docs_result[0]['elementId']
                    }))

        print(graphDB.query(linking_query, 
                    params={
                        'parentId':docs_result[0]['elementId'],
                        'parentType':docs_result[0]['d']['nodeType']
                    }))


        print(graphDB.query(link_to_parent_docs, 
                    params={
                        'parentId':docs_result[0]['elementId']
                    }))

        print(graphDB.query(link_first_to_parent_docs,
                    params={
                        'parentId':docs_result[0]['elementId']
                    }))

        print(graphDB.query(link_docs_to_standard))

    except Exception as e:
        raise e 

In [21]:
def create_basis_nodes(standard,result):
    print(f"Initializing basis nodes for: \n Standard Num: {standard['Standard_num']} \n Standard Title:{standard['Standard_title']} ")


    try:
        basis_result = graphDB.query(basis_query,
                    params={'standardsIn':standard})

        print(graphDB.query(datachunk_query,
                        params={
                            'standardsIn':standard,
                            'chunksIn':result['Basis_for_judgement'],
                            'parentId':basis_result[0]['elementId']
                        }))

        print(graphDB.query(linking_query, 
                        params={
                            'parentId':basis_result[0]['elementId'],
                            'parentType':basis_result[0]['b']['nodeType']
                        }))


        print(graphDB.query(link_to_parent_basis, 
                        params={
                            'parentId':basis_result[0]['elementId']
                        }))

        print(graphDB.query(link_first_to_parent_basis,params={
                'parentId':basis_result[0]['elementId']
            } ))

        print(graphDB.query(link_basis_to_standard))
        
    except Exception as e:
        raise e

In [22]:
def create_def_nodes(standard,result):
    print(f"Initializing definition nodes for: \n Standard Num: {standard['Standard_num']} \n Standard Title:{standard['Standard_title']} ")


    try:
        def_result = graphDB.query(definitions_query,
                    params={'standardsIn':standard})

        print(graphDB.query(datachunk_query,
                        params={
                            'standardsIn':standard,
                            'chunksIn':result['Definitions'],
                            'parentId':def_result[0]['elementId']
                        }))

        print(graphDB.query(linking_query, 
                        params={
                            'parentId':def_result[0]['elementId'],
                            'parentType':def_result[0]['d']['nodeType']
                        }))


        print(graphDB.query(link_to_parent_def, 
                        params={
                            'parentId':def_result[0]['elementId']
                        }))

        print(graphDB.query(link_first_to_parent_def,params={
                'parentId':def_result[0]['elementId']
            } ))

        print(graphDB.query(link_def_to_standard))
        
    except Exception as e:
        raise e

In [ ]:
# Test Large Functions Components
# create_standards_node(standard_1, result) #[pass]
# create_docs_nodes(standard_1,result) #[pass]
# create_basis_nodes(standard_1,result) #[pass]
#create_def_nodes(standard_1, result) #[pass]
 

In [36]:
def process_standards_controller(standard, result):
    create_standards_node(standard,result)
    create_formal_nodes(standard, result)
    create_basis_nodes(standard,result)
    create_docs_nodes(standard,result)
    create_def_nodes(standard,result)

In [ ]:
# Resolving linkin  Issue [RESOLVED]

# standard_1 = json_file[0]
# result = chunk_doc(standard_1)

In [ ]:


# create_standards_node(standard_1, result) #[pass]
# create_docs_nodes(standard_1,result) #[pass]
# create_basis_nodes(standard_1,result) #[pass]
# create_def_nodes(standard_1, result) #[pass]
#process_standards_controller(standard_1,result)

In [ ]:
# standard_2 = json_file[1]
# result_2 = chunk_doc(standard_2)

# for k,v in result_2.items():
#     print(k,v)
# # process_standards_controller(standard_2,result_2)

In [ ]:
# Test Large Functions Components
# create_standards_node(standard_2, result_2) #[pass]
# create_docs_nodes(standard_2,result_2) #[pass]
# create_basis_nodes(standard_2,result_2) #[pass]
# create_def_nodes(standard_2, result_2) #[pass]

In [37]:
#
# LOAD STANDARDS TO DATABASE
#


import time

for standard in json_file:
    result = chunk_doc(standard)
    process_standards_controller(standard,result)
    time.sleep(1)


Initializing standard nodes for: 
 Standard Num: 1 
 Standard Title:Strategic Planning 
Initializing formal nodes for: 
 Standard Num: 1 
 Standard Title:Strategic Planning 
[{'c': {'parentStandardNum': 1, 'standardTitle': 'Strategic Planning', 'nodeCat': 'AACSB', 'text': 'standard 1 strategic planning1.1 school maintains well documented strategic plan developed robust collaborative planning process involving key stakeholder input informs school resource allocation priorities . strategic plan also articulate clear focused mission school . 1.2 school regularly monitors progress planned strategies expected outcomes communicates progress key stakeholders . part monitoring school conducts formal risk analysis plans mitigate identified major risks . 1.3 school carries', 'chunkSeqNum': 0, 'embedding': 0, 'nodeType': 'DATACHUNK', 'UUID': 'f67b2b36-24fa-45af-b482-7c52f01c489a', 'parentId': 196, 'parentType': 'FORMAL'}}, {'c': {'parentStandardNum': 1, 'standardTitle': 'Strategic Planning', 'nod

In [ ]:
result

In [38]:
#
# Load in in AACSB Sections (parent to standards)
#

path =r'..\data\aacsb\processed\SECTIONS_processed_2024-04-04_16-24-04.json'
json_file_section = json.load(open(path))
print(type(json_file_section)) #list

<class 'list'>


In [ ]:
section_1 = json_file_section[0]
section_1


In [39]:
#
# Section Queries
#


section_query = """
MERGE(s:Section {sectionNum: $sectionsIn.Section_Num})
    ON CREATE SET 
        s.title =  $sectionsIn.Section_Title,
        s.nodeType = 'SECTION', 
        s.nodeCat = 'AACSB',
        s.source = 'https://www.aacsb.edu/educators/accreditation/business-accreditation/aacsb-business-accreditation-standards',
        s.year = 2020
RETURN s, ID(s) as elementId
"""

section_description_query ="""
MERGE(c:Chunk {sectionNum: $sectionsIn.Section_Num})
    ON CREATE SET
        c.UUID = apoc.create.uuid(),
        c.title = $sectionsIn.Section_Title,
        c.text = $sectionsIn.Section_Description,
        c.embedding = 0,
        c.nodeType = 'SECTION_DESC',
        c.nodeCat = 'AACSB'
RETURN c, ID(c) as elementId

"""

link_to_parent_section_query=""" 
MATCH (c:Chunk),(s:Section)
WHERE c.sectionNum = s.sectionNum
    AND c.nodeType = 'SECTION_DESC'
    AND s.nodeType = 'SECTION'
MERGE (c)-[newRelationship: DESCIBES]->(s)
RETURN count(newRelationship) AS num_links_created
"""

link_standards_to_parent_sections_query=""" 
MATCH (s: Standard), (p: Section)
WHERE s.parentSectionNum = p.sectionNum
    AND s.nodeCat = 'AACSB'
    AND p.nodeCat = 'AACSB'
MERGE (s)-[newRelationship: BELONGS_TO]->(p)
RETURN count(newRelationship) AS num_links_created
"""


link_sections_to_aacsb_root="""
MATCH (s: Section), (r:Root)
WHERE s.nodeCat = r.nodeCat
    AND r.nodeType = 'ROOT'
    AND r.nodeCat = 'AACSB'
MERGE (s)-[newRelationship: SECTION_OF]->(r)
RETURN count(newRelationship) AS num_links_created
"""

In [40]:
#
# Create Sections and Section Desctiption Nodes
#

def build_section_nodes(section):
    print(f"Initializing section nodes for: \n Section Num: {section['Section_Num']} \n Section Title:{section['Section_Title']} ")


    try:
        graphDB.query("""
            CREATE CONSTRAINT unique_node IF NOT EXISTS 
            FOR (s:Session) REQUIRE s.sectionNum IS UNIQUE
            """)

        graphDB.query(section_query,
                    params ={
                        'sectionsIn': section
                    })

        graphDB.query(section_description_query,
                    params ={
                        'sectionsIn': section
                    })
    except Exception as e:
        raise e

#
# Link Sections and Section Desc Nodes
#
def link_section_and_desc_nodes():
    try:
        return graphDB.query(link_to_parent_section_query)
    except Exception as e:
        raise e



#
# Link Standards to Sections
#

def link_standards_to_sections():
    try:
        return graphDB.query(link_standards_to_parent_sections_query)
    except Exception as e:
        raise e


#
# Link Sections to Root AACSB node
#

def link_sections_to_AACSB_root():
    try:
        return graphDB.query(link_sections_to_aacsb_root)
    except Exception as e:
        raise e
    


In [41]:
for section in json_file_section:
    build_section_nodes(section)

Initializing section nodes for: 
 Section Num: 1 
 Section Title:Strategic Managemend and Innovation 
Initializing section nodes for: 
 Section Num: 2 
 Section Title:Learner Success 
Initializing section nodes for: 
 Section Num: 3 
 Section Title:Though Leadership, Engagement, and Societal Impact 


In [42]:
link_section_and_desc_nodes()

[{'num_links_created': 3}]

In [43]:
link_standards_to_sections()

[{'num_links_created': 9}]

In [46]:
link_sections_to_AACSB_root()

[{'num_links_created': 3}]

In [ ]:
#
# AACSB LOAD IN COMPLETE --> NEXT NOTEBOOK LLM_to_KG.ipynb to LOAD INST Sample Data and KG
#